In [ ]:
import numpy as np


terrenos = [2,1,4,1]

def get_vizinhanca(lote: list[int]) -> np.ndarray:
    """Recebe:
    - Lista com as areas dos terrenos de um lote
    Retorna:
    - Vetor 2 x 2 representando as vizinhancças de cada terreno.
    """
    vizinhanca = []
    vizinhos_direita = np.roll(terrenos, 1)
    vizinhos_esquerda = np.roll(terrenos, -1)
    for i in range(len(lote)):
        vizinhanca.append([[terrenos[i], vizinhos_esquerda[i]], [terrenos[i], vizinhos_direita[i]]])
    return np.array(vizinhanca)

def vizinhos_esquerda_ou_direita(terreno_alvo: int, melhor_adjacente: int) -> int:
    """Recebe:
    terreno_alvo -- O indice do terreno de valor alvo
    melhor_adjacente -- 0 ou 1. Indica a posição em relação ao valor alvo.
        0: posição anterior
        1: posição seguinte
    Retorna:
        Índice do terreno do melhor_adjacente.
    """
    # A função é necessária para tratar dos casos onde o terreno escolhido está nas extremidades,
    # e um simples acréscimo ou decréscimo no índice pode gerar um erro.
    # - Nos casos onde o terreno_alvo é o último elemento do vetor e o melhor_adjacente é o elemento seguinte, vizinho 2 será o elemento 0 do vetor
    # - Nos casos onde o terreno_alvo é o primeiro elemento do vetor e o melhor_adjacente é o elemento anterior, vizinho 2 será o elemento -1 do vetor
    # - Nos demais casos melhor_adjacente será terreno_alvo +1 (seguinte) ou terreno_alvo -1 (anterior)

    if melhor_adjacente == 0 and terreno_alvo < len(terrenos)-1:
        melhor_adjacente = terreno_alvo + 1
    elif melhor_adjacente == 0 and terreno_alvo >= len(terrenos)-1:
        vizinhos2 = 0
    elif melhor_adjacente == 1 and terreno_alvo > 0:
        melhor_adjacente = terreno_alvo - 1
    elif melhor_adjacente == 1 and terreno_alvo <= 0:
        melhor_adjacente = len(terrenos)-1
    return melhor_adjacente

def get_melhor_divisao(vizinhanca: np.ndarray) -> tuple:
    """Recebe:
    vizinhanca -- um vetor 2 x 2 representando os pares de vizinhanças
    Retorna:
    terreno_alvo -- indice do terreno alvo
    melhor_adjacente -- Posição relativa ao terreno_alvo
        0: posição anterior
        1: posição seguinte
    """
    # 1- Pesquisa pelo maior valor de cada par e dentre estes valores identificar o menor, este é o valor alvo.\
    # 2- Pesquisa entre os vizinhos do valor alvo o vizinho de menor área. Estes serão os vizinhos escolhidos.
    # 3- Pesquisa na matriz 2 x 2 o índice do par escolhido.
    # 4- Retorna o indice do valor alvo e 0 (vizinho seguinte) ou 1 (vizinho anterior).

    # Menor area entre maiores areas
    menores_areas = np.amin(vizinhanca, axis=1)
    valor_alvo = np.min(np.amax(menores_areas, axis=1))
    candidatos = np.where((np.max(menores_areas, axis=1) == valor_alvo) & (menores_areas[:,0] == valor_alvo))
    candidato_escolhido = np.amin(vizinhanca[candidatos], axis = 1)[0]
    terreno_alvo, melhor_adjacente = np.where(np.all(vizinhanca == candidato_escolhido, axis=2))
    terreno_alvo, melhor_adjacente = terreno_alvo[0], melhor_adjacente[0]
    # melhor_adjacente só pode ser 0 ou 1 e indica se om melhor terreno para a união está à direita ou esquerda, respectivamente, de terreno_alvo

    print(f'A melhor divisão é entre o terreno {terreno_alvo} e o terreno {melhor_adjacente}')
    return terreno_alvo, melhor_adjacente

def taxar(lote: list, vizinhos: tuple, taxa: float) -> float:
    """Aplica a taxa definida ao terreno_alvo"""
    taxacao = taxa * max([lote[vizinhos[0]], lote[vizinhos[1]]])
    print(f"Colentando {taxacao}")
    return taxacao

def unir_terrenos(vizinhos: tuple, lote: np.ndarray) -> int:
    """Combina o terreno_alvo com o melhor_adjacente"""
    uniao = lote[vizinhos[0]] + lote[vizinhos[1]]
    print(f'A união resulta em um terreno de área {uniao}.')
    return uniao

def atualizar_lote(lote: np.ndarray, terrenos_unidos: tuple, terreno_criado: int) -> np.ndarray:
    """Atualiza o lote dos terrenos originais com os terrenos combinados"""
    return np.insert(np.delete(lote, terrenos_unidos), terrenos_unidos[0], terreno_criado)


In [ ]:
taxa = 1.5
taxa_total = 0
# LOOP
for div in range(len(terrenos)-1):
    lote = get_vizinhanca(terrenos)
    vizinhos = get_melhor_divisao(lote)
    taxa_total += taxar(terrenos, vizinhos, taxa)
    print(f"Taxa total {taxa_total}")
    if div <= len(terrenos):
        novos_terrenos = unir_terrenos(vizinhos, terrenos)
    terrenos = atualizar_lote(terrenos, vizinhos, novos_terrenos)
    print()
    print(terrenos)

print(taxa_total)